In [1]:
"""Descriptive statistics, time series plots, proportional burden calculations, and other analyses."""

import pandas as pd
from statsmodels.stats.weightstats import DescrStatsW
import seaborn as sns
import matplotlib.pyplot as plt

# set matplotlib/seaborn to 300dpi
sns.set_context("paper", font_scale=1.5, rc={"figure.dpi": 300, "lines.linewidth": 2})

# toggle table rounding
rounding = False

# toggle focus on Non-fire and fire - for presentation slides
nofire_fire_only = False

# toggle focus on childs fire vs cmaq fire
childs_cmaq_only = False

In [2]:
df = pd.read_parquet("data/total + wildfire pm and demographic data 6-17-2024.parquet")

# 2007-2018 only for annual statistics
pm_fields = [
    # "PM_total_2006",
    "PM_total_2007",
    "PM_total_2008",
    "PM_total_2009",
    "PM_total_2010",
    "PM_total_2011",
    "PM_total_2012",
    "PM_total_2013",
    "PM_total_2014",
    "PM_total_2015",
    "PM_total_2016",
    "PM_total_2017",
    "PM_total_2018",
    # "PM_total_2019",
    "PM_nofire_2007",
    "PM_nofire_2008",
    "PM_nofire_2009",
    "PM_nofire_2010",
    "PM_nofire_2011",
    "PM_nofire_2012",
    "PM_nofire_2013",
    "PM_nofire_2014",
    "PM_nofire_2015",
    "PM_nofire_2016",
    "PM_nofire_2017",
    "PM_nofire_2018",
    "PM_wf_2007",
    "PM_wf_2008",
    "PM_wf_2009",
    "PM_wf_2010",
    "PM_wf_2011",
    "PM_wf_2012",
    "PM_wf_2013",
    "PM_wf_2014",
    "PM_wf_2015",
    "PM_wf_2016",
    "PM_wf_2017",
    "PM_wf_2018",
    # "wfpm25_childs_2006",
    "wfpm25_childs_2007",
    "wfpm25_childs_2008",
    "wfpm25_childs_2009",
    "wfpm25_childs_2010",
    "wfpm25_childs_2011",
    "wfpm25_childs_2012",
    "wfpm25_childs_2013",
    "wfpm25_childs_2014",
    "wfpm25_childs_2015",
    "wfpm25_childs_2016",
    "wfpm25_childs_2017",
    "wfpm25_childs_2018",
    # "wfpm25_childs_2019",
    # "wfpm25_childs_2020",
]

id_fields = [
    "GISJOIN",
    "state",
    "county",
    "tract",
    "Name",
    "Total Population",
    "Urban Population",
    "Rural Population",
    "Hispanic",
    "NH White",
    "NH Black",
    "NH American Indian and Alaska Native",
    "NH Asian",
    # "NH Native Hawaiian and Other Pacific Islander", 0.1% of pop
    # "NH Other", # 0.2% of pop
    "Income quartile",
    "Language spoken at home: only English",
    "Language other than English spoken at home, speaks English well",
    "Language other than English spoken at home, does not speak English well",
    "state_abbr",
    "EPA Region",
    "NCA Region",
    "RUCA 1",
]

dfp = df.melt(
    id_vars=id_fields,
    value_vars=pm_fields,
    var_name="PM25",
    value_name="Concentration",
)

series_split = dfp["PM25"].str.split("_", expand=True)
dfp["PM dataset"] = series_split[0] + "_" + series_split[1]
dfp["Year"] = series_split[2].astype(int)

# drop childs et al dataset for now
# dfp = dfp[dfp["PM dataset"] != "wfpm25_childs"]

# give the PM dataset a more descriptive name
dfp["PM dataset"] = dfp["PM dataset"].replace(
    {
        "PM_total": "CMAQ Total",
        "PM_nofire": "CMAQ Non-fire",
        "PM_wf": "CMAQ Fire",
        "wfpm25_childs": "CWEE",
    }
)

# use categorical data type to sort the PM datasets
dfp["PM dataset"] = pd.Categorical(
    dfp["PM dataset"],
    ["CMAQ Total", "CMAQ Non-fire", "CMAQ Fire", "CWEE"],
    ordered=True,
)

# same for RUCA 1 categories
dfp["RUCA 1"] = pd.Categorical(
    dfp["RUCA 1"],
    ["Urban core", "Suburban", "Micropolitan", "Small town", "Rural"],
    ordered=True,
)

# same for income quartile
dfp["Income quartile"] = pd.Categorical(
    dfp["Income quartile"],
    [1, 2, 3, 4],
    ordered=True,
)

# replace racial and ethnic group column names
dfp = dfp.rename(
    columns={
        "NH White": "White",
        "NH Black": "Black",
        "NH American Indian and Alaska Native": "AIAN",
        "NH Asian": "Asian",
    }
)

# sort by pm dataset and year
dfp = dfp.sort_values(["PM dataset", "Year"], ascending=[False, True])

if nofire_fire_only:
    # only include cmaq fire and cmaq no fire datasets for presentation figures
    dfp = dfp[dfp["PM dataset"].isin(["CMAQ Non-fire", "CMAQ Fire"])]

if childs_cmaq_only:
    # only include cmaq fire and childs fire datasets for presentation figures
    dfp = dfp[dfp["PM dataset"].isin(["CMAQ Fire", "CWEE"])]

## Population-weighted PM2.5 concentrations, 2007-2018 (µg/m³)

In [3]:
def get_summary_statistics(
    df, weights_col="Total Population", value_col="Concentration"
):
    """Get summary statistics of pop. weighted PM2.5 concentrations."""

    # drop tracts with missing values
    df = df.dropna(subset=[value_col, weights_col])

    # calculate summary statistics
    desc_stats = DescrStatsW(df[value_col], weights=df[weights_col])

    return pd.Series(
        {
            "Mean": desc_stats.mean,
            # "SD": desc_stats.std,
            # "10th percentile": desc_stats.quantile(0.1).values[0],
            # "25th percentile": desc_stats.quantile(0.25).values[0],
            # "50th percentile": desc_stats.quantile(0.5).values[0],
            # "75th percentile": desc_stats.quantile(0.75).values[0],
            # "90th percentile": desc_stats.quantile(0.9).values[0],
        }
    )

In [ ]:
# apply summary statistics function to each group

dfd = (
    dfp.groupby(["PM dataset"])
    .apply(get_summary_statistics)
    .sort_values(["PM dataset"], ascending=[False])
).T
dfd.index = pd.Index([""])

# PM2.5 concentrations by NCA region (µg/m³)
dfreg = (
    dfp.groupby(["NCA Region", "PM dataset"])
    .apply(get_summary_statistics)
    .sort_values(["NCA Region", "PM dataset"], ascending=[True, False])
).unstack()

# PM2.5 concentrations by Rural-Urban Commuting Area (RUCA) classification (µg/m³)# get summary statistics by urban/rural classification
dfr = dfp.groupby(["RUCA 1", "PM dataset"]).apply(get_summary_statistics).reset_index()

dfr = (
    dfr.sort_values(["RUCA 1", "PM dataset"], ascending=[True, False])
    .set_index(["RUCA 1", "PM dataset"])
    .unstack()
)

# create table for each race/ethnicity group

dfre = dfp.melt(
    id_vars=[
        "PM dataset",
        "Concentration",
        "GISJOIN",
        "state",
        "county",
        "tract",
        "Name",
        "Year",
        "Income quartile",
        "Language spoken at home: only English",
        "Language other than English spoken at home, speaks English well",
        "Language other than English spoken at home, does not speak English well",
        "state_abbr",
        "EPA Region",
        "NCA Region",
        "RUCA 1",
    ],
    value_vars=[
        "Hispanic",
        "White",
        "Black",
        "AIAN",
        "Asian",
        # "NH Native Hawaiian and Other Pacific Islander",
        # "NH Other",
    ],
    var_name="Race/ethnicity",
    value_name="Population",
)

# PM2.5 concentrations by racial and ethnic group (µg/m³)

dfres = (
    (
        dfre.groupby(["Race/ethnicity", "PM dataset"]).apply(
            get_summary_statistics, weights_col="Population"
        )
    )
    .sort_values(["Race/ethnicity", "PM dataset"], ascending=[True, False])
    .unstack()
)

# PM2.5 concentrations by language spoken at home (µg/m³)
# create table for each language group

dfl = dfp.melt(
    id_vars=[
        "PM dataset",
        "Concentration",
        "GISJOIN",
        "state",
        "county",
        "tract",
        "Name",
        "Year",
        "Income quartile",
        "state_abbr",
        "EPA Region",
        "NCA Region",
        "RUCA 1",
    ],
    value_vars=[
        "Language spoken at home: only English",
        "Language other than English spoken at home, speaks English well",
        "Language other than English spoken at home, does not speak English well",
    ],
    var_name="Language",
    value_name="Population",
)

dfl["Language"].replace(
    {
        "Language spoken at home: only English": "Only English",
        "Language other than English spoken at home, speaks English well": "Other than English, speaks English well",
        "Language other than English spoken at home, does not speak English well": "Other than English, does not speak English well",
    },
    inplace=True,
)

dfls = (
    (
        dfl.groupby(["Language", "PM dataset"]).apply(
            get_summary_statistics, weights_col="Population"
        )
    )
    .sort_values(["Language", "PM dataset"], ascending=[True, False])
    .unstack()
)

dfls

# get summary statistics by per-capita income quartile
dfpci = (
    dfp.groupby(["Income quartile", "PM dataset"])
    .apply(get_summary_statistics)
    .unstack()
)

# create table of population-weighted mean PM2.5 concentrations (µg/m³)

dfr.columns = dfr.columns.get_level_values(1)
dfres.columns = dfres.columns.get_level_values(1)
dfls.columns = dfls.columns.get_level_values(1)
dfpci.columns = dfpci.columns.get_level_values(1)
dfreg.columns = dfreg.columns.get_level_values(1)


# create table of overall descriptive statistics
table1 = pd.concat(
    [dfd, dfreg, dfr, dfres, dfls, dfpci],
    keys=[
        "Overall",
        "NCA Region",
        "RUCA",
        "Race and ethnicity",
        "Language spoken at home",
        "Income quartile",
    ],
)

# calculate percent of total pm2.5 from wildland fire smoke
table1["CMAQ % Fire"] = table1["CMAQ Fire"] / table1["CMAQ Total"] * 100

# calculate relative burden for each subgroup
for dataset in ["CWEE", "CMAQ Total", "CMAQ Non-fire", "CMAQ Fire"]:
    table1[f"{dataset} Proportional Burden"] = (
        table1[dataset] / table1.loc[("Overall", ""), dataset]
    )

table1_conus = (
    table1.copy()  # save version without updated names or rounding for later regional analysis
)

if rounding:
    # round to 2 significant figures
    from sigfig import round

    table1 = table1.map(lambda x: round(x, sigfigs=2))

# rename columns
table1.columns = [
    "CWEE (µg/m³)",
    "CMAQ Fire (µg/m³)",
    "CMAQ Non-fire (µg/m³)",
    "CMAQ Total (µg/m³)",
    "CMAQ % Fire",
    "CWEE Proportional Burden",
    "CMAQ Total Proportional Burden",
    "CMAQ Non-fire Proportional Burden",
    "CMAQ Fire Proportional Burden",
]

In [5]:
# reorder columns
table1[
    [
        "CMAQ Total (µg/m³)",
        "CMAQ Non-fire (µg/m³)",
        "CMAQ Fire (µg/m³)",
        "CMAQ % Fire",
        "CWEE (µg/m³)",
    ]
].round(1).to_csv(
    "tables/table1.csv"
)  # .to_clipboard()

## Time series of PM2.5 concentrations by NCA Region (µg/m³)

In [ ]:
table1

In [ ]:
# Figure S6

test = (
    dfp.groupby(["PM dataset", "Year", "NCA Region"])
    .apply(get_summary_statistics, weights_col="Total Population")["Mean"]
    .reset_index()
    .dropna()
)

test["PM dataset"] = test["PM dataset"].astype(str)


g = sns.relplot(
    data=test,
    x="Year",
    y="Mean",
    col="PM dataset",
    hue="NCA Region",
    style="NCA Region",
    kind="line",
    height=3,
    aspect=2,
    facet_kws={"sharey": False},
    col_wrap=2,
)


# Remove "PM dataset =" from relplot subplot titles
g.set_titles("{col_name}")
g.set_ylabels("Mean PM$_{2.5}$ (µg/m³)")

# move legend to bottom
sns.move_legend(
    g, "lower center", bbox_to_anchor=(0.45, -0.1), ncol=4, title=None, frameon=True
)

## Time series of PM2.5 concentrations by urban-rural status (µg/m³)

In [ ]:
# Figure S8

dfruca1 = (
    dfp.groupby(["PM dataset", "Year", "RUCA 1"])
    .apply(get_summary_statistics, weights_col="Total Population")["Mean"]
    .reset_index()
    .dropna()
)

dfruca1["PM dataset"] = dfruca1["PM dataset"]


g = sns.relplot(
    data=dfruca1.sort_values(["RUCA 1", "PM dataset"]),
    x="Year",
    y="Mean",
    col="PM dataset",
    hue="RUCA 1",
    style="RUCA 1",
    kind="line",
    height=3,
    aspect=2,
    facet_kws={"sharey": False},
    col_wrap=2,
)


# Remove "PM dataset =" from relplot subplot titles
g.set_titles("{col_name}")
g.set_ylabels("Mean PM$_{2.5}$ (µg/m³)")

# move legend to bottom
sns.move_legend(
    g, "lower center", bbox_to_anchor=(0.45, -0.1), ncol=5, title=None, frameon=True
)

## Time series of PM2.5 concentration by race and ethnicity (µg/m³)

In [ ]:
# Figure S7


dfresy = (
    dfre.groupby(["Race/ethnicity", "PM dataset", "Year"])
    .apply(get_summary_statistics, weights_col="Population")
    .reset_index()
    .dropna()
)
dfresy["PM dataset"] = dfresy["PM dataset"].astype(str)


g = sns.relplot(
    data=dfresy,
    x="Year",
    y="Mean",
    col="PM dataset",
    hue="Race/ethnicity",
    style="Race/ethnicity",
    kind="line",
    height=3,
    aspect=2,
    facet_kws={"sharey": False},
    col_wrap=2,
)

# Remove "PM dataset =" from relplot subplot titles
g.set_titles("{col_name}")
g.set_ylabels("Mean PM$_{2.5}$ (µg/m³)")

# move legend to bottom
sns.move_legend(
    g, "lower center", bbox_to_anchor=(0.45, -0.1), ncol=5, title=None, frameon=True
)

## Time series of PM2.5 concentration by language spoken at home (µg/m³)

In [ ]:
dflsy = (
    dfl.groupby(["Language", "PM dataset", "Year"])
    .apply(get_summary_statistics, weights_col="Population")
    .sort_values("PM dataset")
)

g = sns.relplot(
    data=dflsy.reset_index(),
    x="Year",
    y="Mean",
    col="PM dataset",
    hue="Language",
    style="Language",
    kind="line",
    height=3,
    aspect=2,
    facet_kws={"sharey": False},
    col_wrap=2,
)


# Remove "PM dataset =" from relplot subplot titles
g.set_titles("{col_name}")
g.set_ylabels("Mean PM$_{2.5}$ (µg/m³)")

# move legend to bottom
sns.move_legend(
    g, "lower center", bbox_to_anchor=(0.4, -0.1), ncol=3, title=None, frameon=True
)

## Time series of PM2.5 concentration by Census Tract-level per-capita income quartile (µg/m³)

In [ ]:
# Figure S10

dfppci = dfp.groupby(["Income quartile", "PM dataset", "Year"]).apply(
    get_summary_statistics, weights_col="Total Population"
)
dfppci

g = sns.relplot(
    data=dfppci.reset_index(),
    x="Year",
    y="Mean",
    col="PM dataset",
    hue="Income quartile",
    style="Income quartile",
    kind="line",
    height=3,
    aspect=2,
    facet_kws={"sharey": False},
    col_wrap=2,
)

# Remove "PM dataset =" from relplot subplot titles
g.set_titles("{col_name}")
g.set_ylabels("Mean PM$_{2.5}$ (µg/m³)")

# move legend to bottom
sns.move_legend(
    g,
    "lower center",
    bbox_to_anchor=(0.45, -0.1),
    ncol=5,
    title="Income quartile",
    frameon=True,
)

## Plot relative burden

In [15]:
def plot_relative_burden(
    table1,
    group,
    pm_datasets=[
        "CMAQ Total Proportional Burden",
        "CMAQ Non-fire Proportional Burden",
        "CMAQ Fire Proportional Burden",
    ],
    simple_labels=False,
):
    """Create seaborn catplot for a group"""
    # add "CMAQ Total Proportional Burden", to list to include in the plot
    dfbar = (
        table1.xs(group)[pm_datasets]
        .stack()
        .reset_index()
        .rename(
            columns={
                0: "Proportional Burden",
                "level_1": "PM dataset",
                "level_0": group,
            }
        )
    )
    dfbar.replace("NH American Indian and Alaska Native", "NH AIAN", inplace=True)
    if group == "Race and ethnicity":
        dfbar[group] = dfbar[group].str.replace("NH ", "")

    dfbar["PM dataset"] = dfbar["PM dataset"].str.replace(" Proportional Burden", "")

    if simple_labels:
        dfbar["PM dataset"] = dfbar["PM dataset"].replace(
            {
                "CMAQ Total": "Total PM$_{2.5}$",
                "CMAQ Non-fire": "Non-fire PM$_{2.5}$",
                "CMAQ Fire": "Fire PM$_{2.5}$",
            }
        )

    # add line break into language categories
    if group == "Language spoken at home":
        dfbar[group] = dfbar[group].str.replace(
            "Other than English, ", "Other than English,\n"
        )

    # hack to center on 1.0
    dfbar["Proportional Burden"] = dfbar["Proportional Burden"] - 1

    colors = [
        "#1b9e77",
        "#7570b3",
        "#d95f02",
    ]

    g = sns.catplot(
        data=dfbar,
        kind="bar",
        x=group,
        y="Proportional Burden",
        hue="PM dataset",
        palette=colors if len(pm_datasets) == 3 else colors[:2],
        height=6,
    )

    # rotate x-axis labels
    g.set_xticklabels(rotation=45, ha="right")

    # add horizontal line at 1.0
    g.ax.axhline(0, color="gray", linewidth=1.5, linestyle="-", zorder=0)

    # drop legend title
    g._legend.set_title("")

    # set y axis range
    g.set(ylim=(-0.2, 0.41))

    # add 1 to each y-axis tick label - continuing hack to center on 1.0
    g.set_yticklabels([round(x, 2) for x in g.ax.get_yticks() + 1])

    return g

In [ ]:
# TOC graphic

g = plot_relative_burden(table1, "Race and ethnicity", simple_labels=True)

In [ ]:
g = plot_relative_burden(table1, "Language spoken at home")
# change y axis range to 0.93, 1.02

In [ ]:
g = plot_relative_burden(table1, "Income quartile")

In [ ]:
g = plot_relative_burden(table1, "RUCA")

In [ ]:
# sensitivity analysis
g = plot_relative_burden(
    table1,
    "Race and ethnicity",
    pm_datasets=["CWEE Proportional Burden", "CMAQ Fire Proportional Burden"],
)

In [ ]:
# sensitivity analysis - regional
g = plot_relative_burden(
    table1,
    "NCA Region",
    pm_datasets=["CWEE Proportional Burden", "CMAQ Fire Proportional Burden"],
)

In [ ]:
# relative burden figure for manuscript
# Figure 4

relative_burden_cols = [x for x in table1.columns if "Proportional Burden" in x]

dfbar = (
    table1[relative_burden_cols]
    .stack()
    .reset_index()
    .rename(
        columns={
            0: "Proportional Burden",
            "level_0": "Category",
            "level_1": "Subgroup",
            "level_2": "PM dataset",
        }
    )
)
# exclude overall and regional relative burden
dfbar = dfbar[~dfbar["Category"].isin(["Overall", "NCA Region"])]

# add line break into language categories
dfbar.loc[dfbar.Category == "Language spoken at home", "Subgroup"] = dfbar.loc[
    dfbar.Category == "Language spoken at home", "Subgroup"
].str.replace("Other than English, ", "Other than English,\n")

# hack to center on 1.0
dfbar["Proportional Burden"] = dfbar["Proportional Burden"] - 1

# exclude childs dataset
dfbar = dfbar[~dfbar["PM dataset"].str.contains("CWEE")]

dfbar["PM dataset"] = dfbar["PM dataset"].str.replace(" Proportional Burden", "")

fig, axs = plt.subplots(2, 2, figsize=(12, 10))

colors = [
    "#1b9e77",
    "#7570b3",
    "#d95f02",
]

for i, ax in enumerate(axs.flatten()):
    dfbar_subgroup = dfbar[dfbar["Category"] == dfbar["Category"].unique()[i]]

    fig2 = sns.barplot(
        data=dfbar_subgroup,
        x="Subgroup",
        y="Proportional Burden",
        hue="PM dataset",
        ax=ax,
        legend=False if i != 3 else True,
        palette=colors,
    )
    # set y axis range
    ax.set(ylim=(-0.2, 0.41))
    # drop x axis label
    ax.set_xlabel("")

    # set title to category
    ax.set_title(dfbar["Category"].unique()[i])

    # rotate x-axis labels
    ax.xaxis.set_ticks(ax.get_xticks())
    ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha="right")

    # add horizontal line at 1.0
    ax.axhline(0, color="gray", linewidth=1.5, linestyle="-", zorder=0)


for ax in axs.flatten():
    # add 1 to each y-axis tick label - continuing hack to center on 1.0
    ax.yaxis.set_ticks(ax.get_yticks())
    ax.set_yticklabels([round(x, 2) for x in ax.get_yticks() + 1])
# remove legend
axs.flatten()[3].legend().remove()

# change labels to be more descriptive on ax 3
axs.flatten()[3].set_xticklabels(["1 (lowest)", "2", "3", "4 (highest)"])
axs.flatten()[3].set_xlabel("Income")

# add figure level legend
lines_labels = [ax.get_legend_handles_labels() for ax in fig.axes]
lines, labels = [sum(x, []) for x in zip(*lines_labels)]
fig.legend(lines, labels, loc="center left", bbox_to_anchor=(0.28, 0.05), ncol=3)

fig.tight_layout()

In [ ]:
# relative burden figure for manuscript - childs vs cmaq fire
# Figure S18

relative_burden_cols = [x for x in table1.columns if "Proportional Burden" in x]

dfbar = (
    table1[relative_burden_cols]
    .stack()
    .reset_index()
    .rename(
        columns={
            0: "Proportional Burden",
            "level_0": "Category",
            "level_1": "Subgroup",
            "level_2": "PM dataset",
        }
    )
)
# exclude overall  and regional relative burden
dfbar = dfbar[~dfbar["Category"].isin(["Overall", "NCA Region"])]

# add line break into language categories
dfbar.loc[dfbar.Category == "Language spoken at home", "Subgroup"] = dfbar.loc[
    dfbar.Category == "Language spoken at home", "Subgroup"
].str.replace("Other than English, ", "Other than English,\n")
# hack to center on 1.0
dfbar["Proportional Burden"] = dfbar["Proportional Burden"] - 1

# exclude non fire and total datasets
dfbar = dfbar[
    dfbar["PM dataset"].isin(
        ["CMAQ Fire Proportional Burden", "CWEE Proportional Burden"]
    )
]

dfbar["PM dataset"] = dfbar["PM dataset"].str.replace(" Proportional Burden", "")

fig, axs = plt.subplots(2, 2, figsize=(12, 10))

colors = [
    "#1b9e77",
    "#7570b3",
    # "#d95f02",
]

for i, ax in enumerate(axs.flatten()):
    dfbar_subgroup = dfbar[dfbar["Category"] == dfbar["Category"].unique()[i]]

    fig2 = sns.barplot(
        data=dfbar_subgroup,
        x="Subgroup",
        y="Proportional Burden",
        hue="PM dataset",
        ax=ax,
        legend=False if i != 3 else True,
        palette=colors,
    )
    # set y axis range
    ax.set(ylim=(-0.2, 0.45))
    # drop x axis label
    ax.set_xlabel("")

    # set title to category
    ax.set_title(dfbar["Category"].unique()[i])

    # rotate x-axis labels
    ax.xaxis.set_ticks(ax.get_xticks())
    ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha="right")

    # add horizontal line at 1.0
    ax.axhline(0, color="gray", linewidth=1.5, linestyle="-", zorder=0)


for ax in axs.flatten():
    # add 1 to each y-axis tick label - continuing hack to center on 1.0
    ax.yaxis.set_ticks(ax.get_yticks())
    ax.set_yticklabels([round(x, 2) for x in ax.get_yticks() + 1])


# remove legend
axs.flatten()[3].legend().remove()

# change labels to be more descriptive on ax 3
axs.flatten()[3].set_xticklabels(["1 (lowest)", "2", "3", "4 (highest)"])
axs.flatten()[3].set_xlabel("Income")


# add figure level legend
lines_labels = [ax.get_legend_handles_labels() for ax in fig.axes]
lines, labels = [sum(x, []) for x in zip(*lines_labels)]
fig.legend(lines, labels, loc="center left", bbox_to_anchor=(0.39, 0.05), ncol=3)

fig.tight_layout()

In [ ]:
figdata = (
    table1.loc[
        table1.index.get_level_values(0) == "NCA Region",
        ["CMAQ Fire (µg/m³)", "CWEE (µg/m³)"],
    ]
    .reset_index()
    .rename(columns={"level_1": "Region"})[
        ["Region", "CMAQ Fire (µg/m³)", "CWEE (µg/m³)"]
    ]
    .set_index("Region")
    .stack()
    .reset_index()
)
figdata.level_1 = figdata.level_1.str.replace(" (µg/m³)", "")

fig, ax = plt.subplots(1, 1, figsize=(6, 5))

fig = sns.barplot(
    ax=ax,
    data=figdata.sort_values("level_1", ascending=False),
    x="Region",
    y=0,
    hue="level_1",
    palette=colors[:2],
)
# rotate x-axis labels
ticks = ax.xaxis.set_ticks(ax.get_xticks())
rotated_ticks = ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha="right")

# remove x axis title
ax.set_xlabel("")
# set y axis title
ax.set_ylabel("Pop weighted average PM$_{2.5}$ (µg/m³)")

# move legend to the right
ax.legend(loc="center left", bbox_to_anchor=(1, 0.5))

In [30]:
def table_1_subset_analysis(dfp, rounding=False, table1_conus=table1_conus):
    """Create table 1 for regions and states."""
    # code copied from above and placed into function for use with groupby

    # apply summary statistics function to each group

    dfd = (
        dfp.groupby(["PM dataset"])
        .apply(get_summary_statistics)
        .sort_values(["PM dataset"], ascending=[False])
    ).T
    dfd.index = pd.Index([""])

    # PM2.5 concentrations by NCA region (µg/m³)
    dfreg = (
        dfp.groupby(["NCA Region", "PM dataset"])
        .apply(get_summary_statistics)
        .sort_values(["NCA Region", "PM dataset"], ascending=[True, False])
    ).unstack()

    # PM2.5 concentrations by Rural-Urban Commuting Area (RUCA) classification (µg/m³)# get summary statistics by urban/rural classification
    dfr = (
        dfp.groupby(["RUCA 1", "PM dataset"])
        .apply(get_summary_statistics)
        .reset_index()
    )

    dfr = (
        dfr.sort_values(["RUCA 1", "PM dataset"], ascending=[True, False])
        .set_index(["RUCA 1", "PM dataset"])
        .unstack()
    )

    # create table for each race/ethnicity group

    dfre = dfp.melt(
        id_vars=[
            "PM dataset",
            "Concentration",
            "GISJOIN",
            "state",
            "county",
            "tract",
            "Name",
            "Year",
            "Income quartile",
            "Language spoken at home: only English",
            "Language other than English spoken at home, speaks English well",
            "Language other than English spoken at home, does not speak English well",
            "state_abbr",
            "EPA Region",
            "NCA Region",
            "RUCA 1",
        ],
        value_vars=[
            "Hispanic",
            "White",
            "Black",
            "AIAN",
            "Asian",
            # "NH Native Hawaiian and Other Pacific Islander",
            # "NH Other",
        ],
        var_name="Race/ethnicity",
        value_name="Population",
    )

    # PM2.5 concentrations by racial and ethnic group (µg/m³)

    dfres = (
        (
            dfre.groupby(["Race/ethnicity", "PM dataset"]).apply(
                get_summary_statistics, weights_col="Population"
            )
        )
        .sort_values(["Race/ethnicity", "PM dataset"], ascending=[True, False])
        .unstack()
    )

    # PM2.5 concentrations by language spoken at home (µg/m³)
    # create table for each language group

    dfl = dfp.melt(
        id_vars=[
            "PM dataset",
            "Concentration",
            "GISJOIN",
            "state",
            "county",
            "tract",
            "Name",
            "Year",
            "Income quartile",
            "state_abbr",
            "EPA Region",
            "NCA Region",
            "RUCA 1",
        ],
        value_vars=[
            "Language spoken at home: only English",
            "Language other than English spoken at home, speaks English well",
            "Language other than English spoken at home, does not speak English well",
        ],
        var_name="Language",
        value_name="Population",
    )

    dfl["Language"].replace(
        {
            "Language spoken at home: only English": "Only English",
            "Language other than English spoken at home, speaks English well": "Other than English, speaks English well",
            "Language other than English spoken at home, does not speak English well": "Other than English, does not speak English well",
        },
        inplace=True,
    )

    dfls = (
        (
            dfl.groupby(["Language", "PM dataset"]).apply(
                get_summary_statistics, weights_col="Population"
            )
        )
        .sort_values(["Language", "PM dataset"], ascending=[True, False])
        .unstack()
    )

    # get summary statistics by per-capita income quartile
    dfpci = (
        dfp.groupby(["Income quartile", "PM dataset"])
        .apply(get_summary_statistics)
        .unstack()
    )

    # create table of population-weighted mean PM2.5 concentrations (µg/m³)

    dfr.columns = dfr.columns.get_level_values(1)
    dfres.columns = dfres.columns.get_level_values(1)
    dfls.columns = dfls.columns.get_level_values(1)
    dfpci.columns = dfpci.columns.get_level_values(1)
    dfreg.columns = dfreg.columns.get_level_values(1)

    # create table of overall descriptive statistics
    table1 = pd.concat(
        [dfd, dfreg, dfr, dfres, dfls, dfpci],
        keys=[
            "Overall",
            "NCA Region",
            "RUCA",
            "Race and ethnicity",
            "Language spoken at home",
            "Income quartile",
        ],
    )

    # calculate percent of total pm2.5 from wildland fire smoke
    table1["CMAQ % Fire"] = table1["CMAQ Fire"] / table1["CMAQ Total"] * 100

    # calculate relative burden for each subgroup
    for dataset in ["CWEE", "CMAQ Total", "CMAQ Non-fire", "CMAQ Fire"]:
        table1[f"{dataset} Proportional Burden"] = (
            table1[dataset]
            / table1_conus.loc[
                ("Overall", ""), dataset
            ]  # use table1_conus in denominator to compare to overall conus values
        )

    if rounding:
        # round to 2 significant figures
        from sigfig import round

        table1 = table1.map(lambda x: round(x, sigfigs=2))

    # rename columns
    table1.columns = [
        "CWEE (µg/m³)",
        "CMAQ Fire (µg/m³)",
        "CMAQ Non-fire (µg/m³)",
        "CMAQ Total (µg/m³)",
        "CMAQ % Fire",
        "CWEE Proportional Burden",
        "CMAQ Total Proportional Burden",
        "CMAQ Non-fire Proportional Burden",
        "CMAQ Fire Proportional Burden",
    ]
    return table1.loc[table1.index.get_level_values(0) != "NCA Region"]

In [ ]:
# group dfp by NCA region and apply table_1_subset_analysis

table1_region = (
    dfp.groupby(["NCA Region"])
    .apply(table_1_subset_analysis)
    .reset_index()
    # .drop(columns=["level_1"])
)
table1_region.rename(
    {"level_1": "Category", "level_2": "Subgroup"}, axis=1, inplace=True
)

table1_region["Subgroup"] = table1_region["Subgroup"].astype(str)

region_relative_burden = (
    table1_region.set_index(["NCA Region", "Category", "Subgroup"])[
        [
            "CMAQ Total Proportional Burden",
            "CMAQ Non-fire Proportional Burden",
            "CMAQ Fire Proportional Burden",
            "CWEE Proportional Burden",
        ]
    ]
    .stack()
    .reset_index()
    .rename({"level_3": "Dataset", 0: "Proportional Burden"}, axis=1)
)
region_relative_burden["Dataset"] = region_relative_burden["Dataset"].str.replace(
    " Proportional Burden", ""
)

# drop overall statistics
region_relative_burden = region_relative_burden[
    region_relative_burden["Category"] != "Overall"
]

In [ ]:
# Figure 7

with sns.plotting_context("poster"):

    colors = [
        "#1b9e77",
        "#7570b3",
        "#d95f02",
    ]

    fig, axs = plt.subplots(2, 4, figsize=(22, 14))

    dfbar_region = region_relative_burden.loc[
        (region_relative_burden["NCA Region"].isin(["Southeast", "Northwest"]))
        & (region_relative_burden["Dataset"] != "CWEE")
    ].copy()
    dfbar_region["Category_region"] = (
        dfbar_region["Category"] + " " + dfbar_region["NCA Region"]
    )

    # subtract 1 for hacky bar plot solution (centers on 1, also adjust y axis range later)
    dfbar_region["Proportional Burden"] = dfbar_region["Proportional Burden"] - 1

    for i, ax in enumerate(axs.flatten()):
        dfbar_subgroup = dfbar_region[
            dfbar_region["Category_region"]
            == dfbar_region["Category_region"].unique()[i]
        ]
        print(dfbar_region["Category_region"].unique()[i])
        fig2 = sns.barplot(
            data=dfbar_subgroup,
            x="Subgroup",
            y="Proportional Burden",
            hue="Dataset",
            ax=ax,
            legend=False if i != 3 else True,
            palette=colors,
        )
        # set y axis range
        ax.set(ylim=(-0.7, 1.5))
        # drop x axis label
        ax.set_xlabel("")
        ax.set_ylabel("")

        # drop y axis label
        if i not in [0, 4]:
            ax.set_ylabel("")

        # set title to category
        if i in range(0, 4):
            ax.set_title(dfbar_region["Category"].unique()[i] + "\n ")

        # rotate x-axis labels
        ax.xaxis.set_ticks(ax.get_xticks())
        ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha="right")

        # add horizontal line at 1.0
        ax.axhline(0, color="gray", linewidth=1.5, linestyle="-", zorder=0)

    for ax in axs.flatten():
        # add 1 to each y-axis tick label - continuing hack to center on 1.0

        ax.yaxis.set_major_locator(plt.MaxNLocator(5))

        ax.yaxis.set_ticks(ax.get_yticks())
        ax.set_yticklabels([round(x, 2) for x in ax.get_yticks() + 1])
    # remove legend
    axs.flatten()[3].legend().remove()

    # remove xtick labels for top row
    for i in range(4):
        axs.flatten()[i].set_xticklabels([])

    # remove ytick labels for all columns except leftmost
    for i in range(8):
        if i not in [0, 4]:
            axs.flatten()[i].set_yticklabels([])

    # label regions
    axs.flatten()[0].set_ylabel("Northwest\n", fontsize=24)
    axs.flatten()[4].set_ylabel("Southeast\n", fontsize=24)

    # change labels to be more descriptive on ax 3
    axs.flatten()[7].set_xticklabels(["1 (lowest)", "2", "3", "4 (highest)"])
    # axs.flatten()[7].set_xlabel("Income")

    # add figure level legend
    lines_labels = [ax.get_legend_handles_labels() for ax in fig.axes]
    lines, labels = [sum(x, []) for x in zip(*lines_labels)]
    (fig.legend(lines, labels, loc="center left", bbox_to_anchor=(0.28, 0.0), ncol=3))

    # set title to region
    # fig.suptitle(region)

    # add letter to top left

    # if region == "Southeast":
    #    letter = "A"
    # elif region == "Northwest":
    #    letter = "B"
    # else:
    #    letter = None

    # fig.text(0.05, 0.95, letter, fontsize=16, fontweight="bold") if letter else None

    # set y axis title at fig level
    fig.text(
        -0.02, 0.6, "Proportional Burden", va="center", rotation="vertical", fontsize=32
    )

    fig.tight_layout(h_pad=2, w_pad=0.02)
    plt.show()
    plt.close()

In [ ]:
# Figure 7 - other regions for SI

colors = [
    "#1b9e77",
    "#7570b3",
    "#d95f02",
]

for region in region_relative_burden["NCA Region"].unique():

    dfbar_region = region_relative_burden.loc[
        (region_relative_burden["NCA Region"] == region)
        & (region_relative_burden["Dataset"] != "CWEE")
    ].copy()

    # subtract 1 for hacky bar plot solution (centers on 1, also adjust y axis range later)
    dfbar_region["Proportional Burden"] = dfbar_region["Proportional Burden"] - 1

    fig, axs = plt.subplots(2, 2, figsize=(12, 10))

    for i, ax in enumerate(axs.flatten()):
        dfbar_subgroup = dfbar_region[
            dfbar_region["Category"] == dfbar_region["Category"].unique()[i]
        ]

        fig2 = sns.barplot(
            data=dfbar_subgroup,
            x="Subgroup",
            y="Proportional Burden",
            hue="Dataset",
            ax=ax,
            legend=False if i != 3 else True,
            palette=colors,
        )
        # set y axis range
        ax.set(ylim=(-0.7, 1.5))
        # drop x axis label
        ax.set_xlabel("")

        # set title to category
        ax.set_title(dfbar["Category"].unique()[i])

        # rotate x-axis labels
        ax.xaxis.set_ticks(ax.get_xticks())
        ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha="right")

        # add horizontal line at 1.0
        ax.axhline(0, color="gray", linewidth=1.5, linestyle="-", zorder=0)

    for ax in axs.flatten():
        # add 1 to each y-axis tick label - continuing hack to center on 1.0

        ax.yaxis.set_major_locator(plt.MaxNLocator(5))

        ax.yaxis.set_ticks(ax.get_yticks())
        ax.set_yticklabels([round(x, 2) for x in ax.get_yticks() + 1])
    # remove legend
    axs.flatten()[3].legend().remove()

    # change labels to be more descriptive on ax 3
    axs.flatten()[3].set_xticklabels(["1 (lowest)", "2", "3", "4 (highest)"])
    axs.flatten()[3].set_xlabel("Income")

    # add figure level legend
    lines_labels = [ax.get_legend_handles_labels() for ax in fig.axes]
    lines, labels = [sum(x, []) for x in zip(*lines_labels)]
    (
        fig.legend(
            lines, labels, loc="center left", bbox_to_anchor=(0.28, 0.05), ncol=3
        )
        if region != "Southeast"
        else None
    )

    # set title to region
    fig.suptitle(region)

    # add letter to top left

    if region == "Southeast":
        letter = "A"
    elif region == "Northwest":
        letter = "B"
    else:
        letter = None

    fig.text(0.05, 0.95, letter, fontsize=16, fontweight="bold") if letter else None

    fig.tight_layout()
    plt.show()
    plt.close()

In [ ]:
annual_means = dfp.groupby(["Year", "PM dataset"]).apply(get_summary_statistics)

In [43]:
# annual relative burden time series plot
dfrbts = pd.concat(
    [dfruca1, dfresy, dflsy.reset_index(), dfppci.reset_index()],
    names=[
        "RUCA",
        "Race and ethnicity",
        "Language spoken at home",
        "Income quartile",
    ],
)

In [ ]:
# Figure 6

df_annual_rb = (
    dfrbts.melt(
        id_vars=["PM dataset", "Year", "Mean"],
        value_vars=[
            "RUCA 1",
            "Mean",
            "Race/ethnicity",
            "Language",
            "Income quartile",
        ],
        # value_name="Mean",
    )
    .set_index(["Year", "PM dataset"])
    .join(annual_means, rsuffix="_overall")
).dropna()

# calculate relative burden
df_annual_rb["Proportional Burden"] = (
    df_annual_rb["Mean"] / df_annual_rb["Mean_overall"]
)

df_annual_rb.reset_index(inplace=True)
df_annual_rb.value = df_annual_rb.value.astype(str)

# add new line after comma if comma in value
df_annual_rb["value"] = df_annual_rb["value"].str.replace(", ", ",\n")

# add (lowest) to income quartile 1 and highest to income quartile 4
df_annual_rb["value"] = df_annual_rb["value"].str.replace("1", "1 (lowest)")
df_annual_rb["value"] = df_annual_rb["value"].str.replace("4", "4 (highest)")

# change per-capita income quartile name
df_annual_rb["variable"] = df_annual_rb["variable"].replace(
    "Income quartile", "Income quartile"
)

# change to language spoken at home
df_annual_rb["variable"] = df_annual_rb["variable"].replace(
    "Language", "Language spoken at home"
)
# create a relative burden by year figure

import matplotlib.pyplot as plt

fig, ax = plt.subplots(4, 2, figsize=(16, 12), sharex=True, sharey=True)

axs = ax.flatten()

i = 0

df_annual_rb.variable.replace(
    {"RUCA 1": "RUCA", "Race/ethnicity": "Race and ethnicity"}, inplace=True
)

for variable in df_annual_rb.variable.unique():
    for dataset in ["CMAQ Fire", "CMAQ Non-fire"]:

        plotdf = df_annual_rb.loc[
            (df_annual_rb["PM dataset"] == dataset)
            & (df_annual_rb.variable == variable)
        ].copy()

        if variable == "RUCA":
            # sort
            plotdf["value"] = pd.Categorical(
                plotdf["value"],
                ["Urban core", "Suburban", "Micropolitan", "Small town", "Rural"],
                ordered=True,
            )

        if variable == "Income quartile":
            # also sort
            plotdf["value"] = pd.Categorical(
                plotdf["value"],
                ["1 (lowest)", "2", "3", "4 (highest)"],
                ordered=True,
            )

        sns.lineplot(
            data=plotdf.sort_values(["value"]),
            x="Year",
            y="Proportional Burden",
            hue="value",
            ax=axs[i],
        )

        if i == 0:
            axs[i].set_title("$\\bf{CMAQ\ Fire}$" + f" \n\n {variable}")
        elif i == 1:
            axs[i].set_title("$\\bf{CMAQ\ Non‐fire}$" + f" \n\n {variable}")
        else:
            axs[i].set_title(f"{variable}")

        axs[i].set_ylim(0.5, 1.82)

        axs[i].axhline(1, color="gray", linewidth=1.5, linestyle="-", zorder=0)

        axs[i].set_xlabel("Year")

        axs[i].set_ylabel("Proportional Burden")

        # set y tick locator to 0.1
        axs[i].yaxis.set_major_locator(plt.MaxNLocator(8))

        # remove legend
        axs[i].legend().remove()

        # make legend to the right of figures if i is odd
        if i % 2 != 0:
            lines, labels = axs[i].get_legend_handles_labels()
            _ = axs[i].legend(
                lines,
                labels,
                loc="center left",
                bbox_to_anchor=(1, 0.5),
                ncol=1,
                title=("Income quartile" if variable == "Income quartile" else None),
            )

            # and add text
            # axs[i].text(0,0,variable)

        i += 1

# despine each subplot
for ax in axs:
    sns.despine(ax=ax)

fig.tight_layout()